In [0]:
# Sample DataFrames
data1 = [("Alice", 1), ("Bob", 2), ("Cathy", 3)]
data2 = [("Alice", "HR",1), ("Bob", "IT", 2), ("David", "Finance", 3)]

df1 = spark.createDataFrame(data1, ["name", "id"])
df2 = spark.createDataFrame(data2, ["name", "dept", "id"])

display(df1)
display(df2)


In [0]:
from pyspark.sql.functions import col

# Join
joined_df = df1.join(df2, on="name", how="inner")
display(joined_df)

#joined_df.collect()


In [0]:
from pyspark.sql.functions import sum

# groupBy
grouped_df = df1.groupBy("name").agg(sum("id"))
display(grouped_df)

In [0]:
from pyspark.sql.functions import *

# sort
sorted_df = df1.sort("id", ascending=True)
display(sorted_df)

#select * from table order by id asc, name desc

sorted_df1 = df1.orderBy(desc("id"))
display(sorted_df1)



In [0]:
# repartition
repartitioned_df = df1.repartition(2, "name")
display(repartitioned_df)

#print(df1.rdd.getNumPartitions())
#print(repartitioned_df.rdd.getNumPartitions())

# coalesce
coalesced_df = df1.coalesce(1)
display(coalesced_df)

In [0]:
file = '/Volumes/workspace/default/tmp/country_lookup.csv'

df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(file)
display(df)

In [0]:
repartitioned_df = df.repartition(20)
display(repartitioned_df)

In [0]:
from pyspark.sql.functions import spark_partition_id, sum as spark_sum

# Add partition ID column
df_with_pid = repartitioned_df.withColumn("partition_id", spark_partition_id())

# Count partitions
partition_count = df_with_pid.select("partition_id").distinct().count()

# Calculate size (row count) per partition
partition_sizes = df_with_pid.groupBy("partition_id").count().orderBy("partition_id")

print(f"Partition count: {partition_count}")
display(partition_sizes)

In [0]:
# left Semi join: select rows from df1 with match in df2 on 'id'
semi_left_join_df = df1.join(df2, on="id", how="leftsemi")
display(semi_left_join_df)

# Anti left join: select rows from df1 with no match in df2 on 'id'
anti_left_join_df = df1.join(df2, on="id", how="leftanti")
display(anti_left_join_df)

In [0]:
df1.createOrReplaceTempView("emp")
df2.createOrReplaceTempView("dept")

result = spark.sql("""
SELECT e.name, e.id, d.dept
FROM emp e
LEFT JOIN dept d ON e.name = d.name
ORDER BY e.id DESC
""")

display(result)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank, dense_rank

window_spec = Window.partitionBy("dept").orderBy("id").rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

df_with_window = df2.withColumn("row_num", row_number().over(window_spec)) \
    .withColumn("rank", rank().over(window_spec)) \
    .withColumn("dense_rank", dense_rank().over(window_spec))

display(df_with_window)

In [0]:
df_with_window.write.format("delta").mode("overwrite").saveAsTable("df_with_window_delta")

In [0]:
%sql
select * from workspace.default.df_with_window_delta

In [0]:
spark.sql("show databases").show()
spark.sql("use default").show()
spark.sql("show tables").show()

In [0]:
spark.sql("""
MERGE INTO df_with_window_delta AS target
USING df_with_window AS source
ON target.name = source.name AND target.id = source.id
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED THEN
  INSERT *
""")